In [13]:
import pandas as pd
import streamlit as st
import plotly.express as px
from PIL import Image
import os
import camelot
import matplotlib.pyplot as plt
import numpy as np
import base64
import io
import glob
import plotly.offline as pyo
import plotly.graph_objs as go
import streamlit.components.v1 as components
import re


tables = camelot.read_pdf(f'../statements/dbs_statement_2.pdf', flavor='stream', pages='all')
concat = tables[0].df

for i in range(1, len(tables)):
    concat = pd.concat([concat, tables[i].df])

concat = concat.drop(index=0)
concat = concat.drop(index=1)
concat = concat.reset_index(drop=True)

if concat.shape[1] == 7:
    concat.columns = ['Date', 'Extra', 'Code', 'Reference', 'Debit', 'Extra2', 'Credit']
    concat = concat.drop(columns=['Extra'])
else:
    concat.columns = ['Date', 'Code', 'Reference', 'Debit', 'Extra2', 'Credit']

concat = concat.drop(index=0)
concat = concat.drop(index=1)


concat.columns = ['Date', 'Code', 'Reference', 'Debit', 'Extra', 'Credit']
concat = concat.reset_index(drop=True)

for row in range(len(concat)):
    debit = concat.at[row, 'Debit']
    credit = concat.at[row, 'Credit']
    extraVal = concat.at[row, 'Extra']

    if debit == '' and credit != '':
        concat.at[row, 'Debit'] = credit
        concat.at[row, 'Credit'] = ''
    
    if credit == '' and extraVal != '':
        concat.at[row, 'Credit'] = extraVal
        concat.at[row, 'Extra'] = ''

concat = concat.drop(columns=['Extra'])

for a in range(len(concat)):
    code = concat.at[a, 'Code']
    steps = 0
    
    if code != '':
        addArr = []
        index = a + 1
        
        while True:
            if index > len(concat)-1:
                break
                
            check = concat.at[index, 'Code']
            if check == '':
                addArr.append(concat.at[index, 'Reference'])
                steps += 1
                index += 1
            else:
                break
            
        for step in range(steps):
            if concat.at[a, 'Reference'][-1] == '-' or concat.at[a, 'Reference'][-1].isdigit() == True:
                concat.at[a, 'Reference'] += concat.at[a+step+1, 'Reference']
            else:
                concat.at[a, 'Reference'] += ' ' + concat.at[a+step+1, 'Reference']
                
            concat.at[a+step+1, 'Reference'] = ''

## remove all empty rows
concat = concat.loc[~(concat == '').all(axis=1)]
concat = concat.reset_index(drop=True)

for TYPE in range(len(concat)):
    code = concat.at[TYPE, 'Code']
    
    if code == 'MST':
        concat.at[TYPE, 'Type'] = 'Card Transaction'
        
    if code == 'ITR' or code == 'ICT':
        concat.at[TYPE, 'Type'] = 'Funds Transfer'
        
    if code == 'POS':
        concat.at[TYPE, 'Type'] = 'Point-Of-Sale'
        
    if code == 'INT':
        concat.at[TYPE, 'Type'] = 'Interest Earned'

    if code == 'ADV':
        concat.at[TYPE, 'Type'] = 'PayLah'

concat = concat.reindex(index=concat.index[::-1])
concat.reset_index(drop=True)

concat = concat.loc[~((concat['Date'] == 'Date') & (concat['Code'] == 'Code'))]
concat = concat.loc[~((concat['Debit'] == 'Download') & (concat['Credit'] == 'Print'))]
concat = concat.loc[~((concat['Debit'] == '(Withdrawal)') & concat['Credit'] == '(Deposit)')]
concat = concat.replace('', np.nan)
concat = concat.dropna(thresh=len(concat.columns) - 3)
concat = concat.reset_index(drop=True)
concat = concat.loc[~((concat['Date'].str.contains('Transaction History')))]
concat = concat.reset_index(drop=True)
concat = concat.reindex(columns=['Date', 'Code', 'Type', 'Reference', 'Debit', 'Credit'])

concat.to_csv('st2.csv')

2023-02-27T23:36:39 - INFO - Processing page-1
2023-02-27 23:36:39.729 INFO    camelot: Processing page-1
2023-02-27T23:36:40 - INFO - Processing page-2
2023-02-27 23:36:40.324 INFO    camelot: Processing page-2
2023-02-27T23:36:40 - INFO - Processing page-3
2023-02-27 23:36:40.999 INFO    camelot: Processing page-3
2023-02-27T23:36:41 - INFO - Processing page-4
2023-02-27 23:36:41.538 INFO    camelot: Processing page-4
2023-02-27T23:36:41 - INFO - Processing page-5
2023-02-27 23:36:41.643 INFO    camelot: Processing page-5
